Example / Test Main

In [26]:
%reload_ext autoreload
%autoreload 2

In [27]:
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
import numpy as np

import os
import cProfile
from tqdm import tqdm

In [28]:
from gde_utils.date_utils import to_datetime
from database_io import DB_player, DB_player_age
from game.game_timeline import GameTimeline

In [29]:
ws = sd.WhoScored(
    #  "GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", "FRA-Ligue 1", "ITA-Serie A"
    leagues=["GER-Bundesliga2", "GER-Bundesliga"],
    seasons=[17, 18, 19, 20, 21],
    # leagues=["GER-Bundesliga"],
    # seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

Starting ChromeDriver in headless mode. 4.11.2


[01/19/24 13:47:34] INFO     patching driver executable                                              ]8;id=689430;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/undetected_chromedriver/patcher.py\patcher.py]8;;\:]8;id=419468;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/undetected_chromedriver/patcher.py#346\346]8;;\
                             /home/morten/.local/share/undetected_chromedriver/undetected_chromedriv               
                             er                                                                                    

In [30]:
schedule = ws.read_schedule().reset_index()
test_game = schedule["game_id"].values[0]
test_date = schedule[schedule["game_id"] == test_game]["date"].values[0]

In [31]:
schedule = schedule.sort_values("date")

In [32]:
schedule

,league,season,game,date,home_team,away_team,game_id,url,stage
1530,GER-Bundesliga2,1718,2017-07-28 Bochum-St. Pauli,2017-07-28 19:30:00,Bochum,St. Pauli,1202134,https://www.whoscored.com/Matches/1202134/Live...,NaN
1533,GER-Bundesliga2,1718,2017-07-29 Ingolstadt-Union Berlin,2017-07-29 12:00:00,Ingolstadt,Union Berlin,1202139,https://www.whoscored.com/Matches/1202139/Live...,NaN
1531,GER-Bundesliga2,1718,2017-07-29 Arminia Bielefeld-Jahn Regensburg,2017-07-29 14:30:00,Arminia Bielefeld,Jahn Regensburg,1202136,https://www.whoscored.com/Matches/1202136/Live...,NaN
1532,GER-Bundesliga2,1718,2017-07-29 Darmstadt-Greuther Fuerth,2017-07-29 14:30:00,Darmstadt,Greuther Fuerth,1202135,https://www.whoscored.com/Matches/1202135/Live...,NaN
1536,GER-Bundesliga2,1718,2017-07-30 Nuernberg-Kaiserslautern,2017-07-30 12:30:00,Nuernberg,Kaiserslautern,1202138,https://www.whoscored.com/Matches/1202138/Live...,NaN
...,...,...,...,...,...,...,...,...,...
3053,GER-Bundesliga2,2122,2022-05-15 FC Heidenheim-Karlsruher SC,2022-05-15 14:30:00,FC Heidenheim,Karlsruher SC,1557925,https://www.whoscored.com/Matches/1557925/Live...,NaN
3052,GER-Bundesliga2,2122,2022-05-15 Dynamo Dresden-Erzgebirge Aue,2022-05-15 14:30:00,Dynamo Dresden,Erzgebirge Aue,1557934,https://www.whoscored.com/Matches/1557934/Live...,NaN
3058,GER-Bundesliga2,2122,2022-05-15 St. Pauli-Fortuna Duesseldorf,2022-05-15 14:30:00,St. Pauli,Fortuna Duesseldorf,1557926,https://www.whoscored.com/Matches/1557926/Live...,NaN
3054,GER-Bundesliga2,2122,2022-05-15 Hannover-Ingolstadt,2022-05-15 14:30:00,Hannover,Ingolstadt,1557928,https://www.whoscored.com/Matches/1557928/Live...,NaN


In [34]:
import logging
logger = logging.getLogger()
logger.disabled = True
dbh_player = DB_player("/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/GDE.db")
dbh_player_age = DB_player_age("/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/GDE.db")
schedule = ws.read_schedule().reset_index()
schedule = schedule.sort_values("date")
for league, game, date in zip(schedule["league"].values, schedule["game_id"].values, schedule["date"].values):
    date = to_datetime(date)
    game_timeline = GameTimeline(ws, game, date, league, dbh_player, dbh_player_age)
    game_timeline.handle()

/home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/clubelo.py:18: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(
/home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/clubelo.py:18: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(
/home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/clubelo.py:18: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the 

___